<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Multi_Agent_Solution_Code_GPT5_GROK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Core LangChain packages
!pip install langchain langchain-community -q

# LLM integrations for OpenAI and xAI
!pip install langchain-openai -q

# NOTE: The package for xAI/Grok integration in LangChain is called 'langchain-xai'.
# Install this package to use Grok models.
!pip install langchain-xai -q

# Search tool for the Research Agent
!pip install duckduckgo-search -q

In [ ]:
!pip uninstall duckduckgo-search -y
!pip install ddgs

In [5]:
import warnings
warnings.filterwarnings("ignore")

# --- Install necessary packages first ---
# This ensures all required libraries are available before importing them.
!pip install langchain langchain-community -q
!pip install langchain-openai -q
!pip install langchain-xai -q # Ensure langchain-xai is installed
!pip install ddgs -q

# --- Import modules after installation ---
import os
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_xai import ChatXAI # This import will now happen after the package is installed
from ddgs import DDGS  # Import the DDGS class

import warnings
warnings.filterwarnings("ignore")


# --- 1. Setup and LLM Configuration ---
# Configuration for API keys
XAI_API_KEY = None
OPENAI_API_KEY = None

try:
    from google.colab import userdata
    XAI_API_KEY = userdata.get('XAI_KEY')
    if not XAI_API_KEY:
        print("WARNING: XAI_KEY not found in Colab secrets.")

    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        print("WARNING: OPENAI_API_KEY not found in Colab secrets.")

except ImportError:
    print("WARNING: Not in Colab. Attempting to get API keys from environment variables.")
    XAI_API_KEY = os.environ.get('XAI_KEY', None)
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', None)

# Define the LLMs to use
llms_to_use = {}

if OPENAI_API_KEY:
    # Research LLM uses the GPT-5 model.
    llms_to_use['research_llm'] = ChatOpenAI(model="gpt-5", api_key=OPENAI_API_KEY, temperature=1.0)
else:
    print("FATAL ERROR: OPENAI_API_KEY not set. The research LLM cannot be initialized.")
    exit()

if XAI_API_KEY:
    # Writing Agent uses the Grok-4 model for content generation.
    llms_to_use['writer_llm'] = ChatXAI(model="grok-4-0709", xai_api_key=XAI_API_KEY, temperature=0.7)
else:
    print("WARNING: XAI_API_KEY not set. Only using GPT for the writing agent as a fallback.")
    # Fallback to OpenAI if Grok key is missing
    llms_to_use['writer_llm'] = ChatOpenAI(model="gpt-5", api_key=OPENAI_API_KEY, temperature=1.0)


# Modified print statement to show configured LLMs
print(f"LLMs configured successfully: Research LLM ({llms_to_use['research_llm'].model_name}), Writer LLM ({llms_to_use['writer_llm'].model_name}).")

# --- 2. Define Agents and Tools ---
print("\nDefining agents and tools...")

# Tool for the Research Agent using DDGS
def ddgs_search(query: str) -> str:
    """Runs a search using DDGS and returns the results."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=5)]
        return "\n".join(results)

# Since initialize_agent with CHAT_CONVERSATIONAL_REACT_DESCRIPTION was causing issues with 'stop' parameter,
# we will create a custom research chain that incorporates the search tool.
# This approach avoids the automatic 'stop' parameter injection by initialize_agent.

# Define a prompt template for the research chain
research_prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Perform a detailed search for the following query and summarize the key findings:\n\nQuery: {query}\n\nSearch Results:"
)

# Create the research chain. This chain will first use the DDGS tool and then summarize results with GPT-5.
# Note: This is a simplified approach. For more complex agentic behavior (e.g., iterative tool use),
# you might need a custom LangChain agent executor or a more advanced chain.
class ResearchChain(LLMChain):
    def _call(self, inputs):
        query = inputs["query"]
        # Use the search tool directly
        search_results = ddgs_search(query)
        # Now, pass the search results to the LLM for summarization
        llm_input = f"{research_prompt_template.template.format(query=query)}\n{search_results}"
        response = self.llm.invoke(llm_input)
        return {"text": response.content} # Ensure output key matches expected by next chain

research_chain = ResearchChain(llm=llms_to_use['research_llm'], prompt=research_prompt_template)


# Writing Agent: Uses Grok-4 to write based on research
writer_prompt_template = PromptTemplate(
    input_variables=["research_result"],
    template="You are an expert article writer. Your task is to write a concise, professional article based on the following research data. The article should be a maximum of 3 paragraphs.\n\nResearch Data:\n{research_result}\n\nArticle:"
)
writer_chain = LLMChain(llm=llms_to_use['writer_llm'], prompt=writer_prompt_template)
print(f"Agents defined successfully: ResearchChain, Writer_chain.")

# --- 3. Orchestrate the Workflow ---
print("\nStarting the multi-agent workflow...")

# Run the workflow with a user query
user_query = "What is quantum computing and how does it differ from classical computing?" # Updated the user query

# Execute the research chain first
print("Researching topic...")
# The research_chain expects 'query' as input
research_output_dict = research_chain.invoke({"query": user_query})
research_output = research_output_dict['text'] # Extract the text content
print("Research complete.")

# Pass the research output to the writing chain
print("Generating article...")
final_article = writer_chain.run(research_result=research_output)
print("Article generation complete.")

print("\n--- Final Article Output ---")
print(final_article)


LLMs configured successfully: Research LLM (gpt-5), Writer LLM (grok-4-0709).

Defining agents and tools...
Agents defined successfully: ResearchChain, Writer_chain.

Starting the multi-agent workflow...
Researching topic...
Research complete.
Generating article...
Article generation complete.

--- Final Article Output ---
### Understanding Quantum Computing: A Paradigm Shift from Classical Methods

Quantum computing represents a revolutionary approach to computation that harnesses the principles of quantum mechanics, such as superposition, entanglement, and interference, to process information. Unlike classical computers, which rely on bits that exist strictly as 0 or 1, quantum computers use qubits that can occupy a superposition of states, allowing an n-qubit system to represent information in a 2^n-dimensional space. This enables operations that manipulate probability amplitudes through unitary, reversible gates, leading to interference patterns that amplify correct solutions while